In [7]:
from datasets import load_dataset

# Load a dialogue dataset (e.g., DailyDialog)
dataset = load_dataset("daily_dialog", trust_remote_code=True)

# Example: Print a sample conversation
print(dataset["train"][0])

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ', " Good.Let ' s go now . ", ' All right . '], 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4], 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4]}


In [8]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_function(examples):
    dialogues = examples["dialog"]
    inputs = []
    outputs = []

    # Convert each conversation into input-output pairs
    for dialog in dialogues:
        for i in range(len(dialog) - 1):  # Ignore last turn (no response)
            inputs.append("chatbot: " + dialog[i])  # User input
            outputs.append(dialog[i + 1])  # Expected chatbot response

    # Tokenize input and output
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["dialog", "act", "emotion"])

# Check processed sample
print(tokenized_datasets["train"][0])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


{'input_ids': [3582, 4045, 10, 10403, 3, 6, 6006, 3, 6, 149, 81, 352, 21, 3, 9, 360, 36, 277, 227, 2634, 3, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [148, 214, 24, 19, 24873, 68, 19, 310, 59, 207, 21, 69, 4639, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [58]:
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
from transformers import T5ForConditionalGeneration

# Load pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [3]:
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [5]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./chatbot_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,  # Adjust based on memory
    per_device_eval_batch_size=4,
    num_train_epochs=2,  # Reduce for testing
    learning_rate=3e-5,
    logging_steps=100,  # Less logging for speed
    save_total_limit=1,  # Keep only last checkpoint
    fp16=True,  # Mixed precision
    gradient_checkpointing=True,  # Memory optimization
    report_to="none",
)


In [9]:
from transformers import Trainer

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [10]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,0.407800,0.388080
2,0.411700,0.383672


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


TrainOutput(global_step=38026, training_loss=0.4348854449561122, metrics={'train_runtime': 4776.7182, 'train_samples_per_second': 31.843, 'train_steps_per_second': 7.961, 'total_flos': 5146507342774272.0, 'train_loss': 0.4348854449561122, 'epoch': 2.0})

In [16]:
# Save the trained model and tokenizer
model.save_pretrained("./chatbot_results")
tokenizer.save_pretrained("./chatbot_results")

('./chatbot_results/tokenizer_config.json',
 './chatbot_results/special_tokens_map.json',
 './chatbot_results/spiece.model',
 './chatbot_results/added_tokens.json')

In [17]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

# Load tokenizer and model from saved checkpoint
tokenizer = AutoTokenizer.from_pretrained("./chatbot_results")
model = T5ForConditionalGeneration.from_pretrained("./chatbot_results")

# Set model to evaluation mode
model.eval()

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [20]:
import torch
import math

def compute_perplexity(model, tokenizer, dataset):
    model.eval()
    total_loss = 0
    total_count = 0

    for example in dataset["test"]:  # Evaluate on test set
        input_text = "chatbot: " + " ".join(example["dialog"])  # Fix: Convert list to string
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss.item()
            total_loss += loss
            total_count += 1

    avg_loss = total_loss / total_count
    perplexity = math.exp(avg_loss)
    return perplexity

ppl = compute_perplexity(model, tokenizer, dataset)
print(f"Perplexity: {ppl:.2f}")  # Lower is better

Perplexity: 1.90


In [22]:
!pip install sacrebleu rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.2 MB/s eta 0:00:00


In [24]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load your fine-tuned model
model_path = "./chatbot_results"  # Ensure your model is saved here
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
model.eval()  # Set model to evaluation mode

# Function to generate chatbot responses
def chatbot_response(user_input):
    input_text = f"chatbot: {user_input}"  # Custom prompt style
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    output = model.generate(**inputs, max_length=100, num_beams=3, early_stopping=True)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

In [32]:
print("Chatbot is ready! Type 'exit' to end the conversation.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye! Have a great day! 😊")
        break

    response = chatbot_response(user_input)
    print(f"Chatbot: {response}\n")

Chatbot is ready! Type 'exit' to end the conversation.

You: hello
Chatbot: do you have any questions?

You: how is weather
Chatbot: it's rainy and wet.

You: exit
Chatbot: Goodbye! Have a great day! 😊
